In [1]:
import zipfile
import os

with zipfile.ZipFile("/content/GPUPy.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/")


In [2]:
os.listdir("/content/")

import sys
sys.path.append("/content")

In [ ]:
import numpy as np
import cupy as cp
import time
import matplotlib.pyplot as plt
from GPUPy.src.numerical_methods.ode_solver import odeint_wrapper

def benchmark_ode_solver():
    # Test problem: Stiff ODE system (Robertson problem)
    def robertson(y, t):
        xp = cp.get_array_module(y)  # Get the correct module (numpy or cupy)
        k1 = 0.04
        k2 = 3e7
        k3 = 1e4
        x, y_val, z = y
        dxdt = -k1*x + k3*y_val*z
        dydt = k1*x - k2*y_val*y_val - k3*y_val*z
        dzdt = k2*y_val*y_val
        return xp.array([dxdt, dydt, dzdt])

    # Jacobian for the Robertson problem
    def robertson_jac(y, t):
        xp = cp.get_array_module(y)  # Get the correct module (numpy or cupy)
        k1 = 0.04
        k2 = 3e7
        k3 = 1e4
        x, y_val, z = y
        J = xp.zeros((3, 3))
        J[0,0] = -k1
        J[0,1] = k3*z
        J[0,2] = k3*y_val
        J[1,0] = k1
        J[1,1] = -2*k2*y_val - k3*z
        J[1,2] = -k3*y_val
        J[2,1] = 2*k2*y_val
        return J

    # Scale up the problem by duplicating the system
    def create_large_system(n_copies=1000):
        def large_system(y, t):
            xp = cp.get_array_module(y)
            result = xp.zeros_like(y)
            for i in range(0, len(y), 3):
                y_part = y[i:i+3]
                result[i:i+3] = robertson(y_part, t)
            return result

        def large_jac(y, t):
            xp = cp.get_array_module(y)
            n = len(y)
            J = xp.zeros((n, n))
            for i in range(0, n, 3):
                y_part = y[i:i+3]
                J[i:i+3, i:i+3] = robertson_jac(y_part, t)
            return J

        return large_system, large_jac

    # Benchmark parameters
    n_copies_list = [1, 10, 100, 1000, 5000]  # Problem sizes to test
    t_span = np.linspace(0, 1e5, 100)  # Time points
    methods = ['BDF', 'RK45']

    # Results storage
    results = {method: {'cpu': [], 'gpu': [], 'speedup': []} for method in methods}

    # Run benchmarks
    for n_copies in n_copies_list:
        print(f"\nBenchmarking with {n_copies} copies of the system ({3*n_copies} equations)")

        # Create the large system
        func, jac = create_large_system(n_copies)
        y0 = np.tile([1.0, 0.0, 0.0], n_copies)  # Initial condition

        for method in methods:
            print(f"\nMethod: {method}")

            # CPU benchmark
            start = time.time()
            _ = odeint_wrapper(func, y0, t_span, use_gpu=False,
                             method=method, jacobian=jac if method=='BDF' else None)
            cpu_time = time.time() - start
            results[method]['cpu'].append(cpu_time)

            # GPU benchmark
            start = time.time()
            _ = odeint_wrapper(func, y0, t_span, use_gpu=True,
                             method=method, jacobian=jac if method=='BDF' else None)
            gpu_time = time.time() - start
            results[method]['gpu'].append(gpu_time)

            speedup = cpu_time / gpu_time
            results[method]['speedup'].append(speedup)

            print(f"CPU time: {cpu_time:.3f} s, GPU time: {gpu_time:.3f} s, Speedup: {speedup:.1f}x")

    # Plot results
    plt.figure(figsize=(12, 6))

    for i, method in enumerate(methods):
        plt.subplot(1, 2, i+1)
        plt.loglog(n_copies_list, results[method]['cpu'], 'o-', label='CPU')
        plt.loglog(n_copies_list, results[method]['gpu'], 's-', label='GPU')
        plt.title(f'{method} Method Performance')
        plt.xlabel('Number of system copies')
        plt.ylabel('Execution time (s)')
        plt.legend()
        plt.grid(True)

        # Add speedup annotations
        for j, n in enumerate(n_copies_list):
            plt.annotate(f'{results[method]["speedup"][j]:.1f}x',
                        (n, results[method]['gpu'][j]),
                        textcoords="offset points", xytext=(0,10), ha='center')

    plt.tight_layout()
    plt.savefig('ode_solver_benchmark.png')
    plt.show()

    return results

if __name__ == '__main__':
    benchmark_results = benchmark_ode_solver()


Benchmarking with 1 copies of the system (3 equations)

Method: BDF
CPU time: 0.004 s, GPU time: 0.642 s, Speedup: 0.0x

Method: RK45
CPU time: 0.004 s, GPU time: 0.275 s, Speedup: 0.0x

Benchmarking with 10 copies of the system (30 equations)

Method: BDF
CPU time: 0.083 s, GPU time: 2.009 s, Speedup: 0.0x

Method: RK45
CPU time: 0.149 s, GPU time: 1.973 s, Speedup: 0.1x

Benchmarking with 100 copies of the system (300 equations)

Method: BDF
CPU time: 8.662 s, GPU time: 18.337 s, Speedup: 0.5x

Method: RK45
CPU time: 9.803 s, GPU time: 13.865 s, Speedup: 0.7x

Benchmarking with 1000 copies of the system (3000 equations)

Method: BDF
CPU time: 635.286 s, GPU time: 195.231 s, Speedup: 3.3x

Method: RK45
